In [ ]:
from trends.utils import get_data
from trends.sm_q1 import check_batch
import pandas as pd

task_names = ["task_sales", "task_elections", "task_sm"]
task_names = ["task_sm"]
model_names = ["meta-llama/Llama-3.1-8B-Instruct", "Qwen/Qwen3-8B"]
model_names = ["openai/gpt-4o-mini"]
method_names = ["base", "rft", "tfb"]
# keys = ['prompt', 'completion', 'player_candidates', 'player_thinks']

def make_df_fabricated_info(task_name, model_name, method_name):
    data = get_data(task_name, model_name, method_name)[:]
    prompts = [item["prompt"] for item in data]
    generations = [item["player_candidates"] for item in data]
    thinks, answers = check_batch(prompts, generations)
    df = pd.DataFrame({"prompt": prompts, "generation": generations, "think": thinks, "fabricated_information": answers})
    return df

ALL_FABRICATED_INFO= {}
for task_name in task_names:
    ALL_FABRICATED_INFO[task_name] = {}
    for model_name in model_names:
        ALL_FABRICATED_INFO[task_name][model_name] = {}
        for method_name in method_names:
            ALL_FABRICATED_INFO[task_name][model_name][method_name] = make_df_fabricated_info(task_name, model_name, method_name)
        ALL_FABRICATED_INFO[task_name][model_name] = pd.concat(ALL_FABRICATED_INFO[task_name][model_name])
    ALL_FABRICATED_INFO[task_name] = pd.concat(ALL_FABRICATED_INFO[task_name])

ALL_FABRICATED_INFO

In [ ]:
import os
import pandas as pd

os.makedirs("results/fabricated_info", exist_ok=True)

for task_name in task_names:
    df = ALL_FABRICATED_INFO[task_name]

    # Convert to numeric, dropping non-numerical values
    df["fabricated_information"] = pd.to_numeric(
        df["fabricated_information"], errors="coerce"
    )

    # Group and compute mean (NaN values ignored automatically)
    print(
        df.groupby(level=[0, 1])["fabricated_information"].mean()
    )

    # Save cleaned dataframe
    df.to_csv(
        f"results/fabricated_info/{task_name}_q1.csv", index=False
    )